In [1]:
import sqlite3
import json
from datetime import datetime
import time

In [2]:
timeframe = '2015-01'
start_row = 0
cleanup = 1000000
sql_transaction =[]
conn = sqlite3.connect('{}.db'.format(timeframe))
c= conn.cursor()

In [3]:
def create_table():
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply(
    parent_id TEXT PRIMARY KEY,
    comment_id TEXT UNIQUE,
    parent TEXT,
    comment TEXT,
    subreddit TEXT,
    unix INT,
    score INT)""")
    

In [4]:
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        conn.commit()
        sql_transactiom =[]
    

In [5]:
def format_data(data):
    data = data.replace('\n','newlinechar').replace('\r', 'newlinechar').replace('"', "'")
    return data

In [6]:
def find_parent(pid):
    try:
        c.execute("""SELECT comment
                    FROM parent_reply
                    WHERE comment_id = '{}' LIMIT 1""".format(pid))
        result = c.fetchone()
        if result!=None:
            return result[0]
        else:
            return False
    except Exception as e:
            #print(str(e))
            return False
        

In [7]:
def find_existing_comment_score(pid):
    try:
        c.execute("""SELECT score
                    FROM parent_reply
                    WHERE parent_id = '{}' LIMIT 1""".format(pid))
        result = c.fetchone()
        if result!= None:
            return result[0]
        else:
            return False
    except Exception as e:
        #print(str(e))
            return False
    

In [8]:
def acceptable_comment(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

In [9]:
def sql_insert_replace_comment(commentid, parentid, parent, comment, subreddit, time, score):
    try:
        sql = """UPDATE parent_reply
                SET parent_id = ?,
                    comment_id = ?,
                    parent_name = ?,
                    parent = ?,
                    comment = ?,
                    subreddit = ?,
                    unix = ?,
                    score = ?
                    WHERE parent_id = ?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion', str(e))
        
    

In [10]:
def sql_insert_has_parent(commentid, parentid, parent, comment, subreddit, time, score):
    try:
        sql = """INSERT INTO parent_reply(
                        parent_id,
                        comment_id,
                        parent,
                        comment,
                        subreddit,
                        unix,
                        score)
                 VALUES ("{}","{}","{}","{}","{}","{}","{}");""".format(parentid,commentid,parent,comment,subreddit,int(time),score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion', str(e))

In [11]:
def sql_insert_no_parent(commentid, parentid, comment, subreddit, time, score):
    try:
        sql = """INSERT INTO parent_reply(
                        parent_id,
                        comment_id,
                        comment,
                        subreddit,
                        unix,
                        score)
                 VALUES ("{}","{}","{}","{}","{}","{}");""".format(parentid,commentid,comment,subreddit,int(time),score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion', str(e))

In [ ]:
if __name__ == '__main__':
    create_table()
    row_cnt =0;
    paired_rows =0;
    with open('RC_{}'.format(timeframe), buffering =1000) as f:
        for row in f:
            row_cnt+=1
            
            if row_cnt > start_row:
                try:
                    row = json.loads(row)
                    parent_id = row['parent_id'].split('_')[1]
                    body = format_data(row['body'])
                    created_utc = row['created_utc']
                    score = row['score']
                    comment_id = row['id']
                    subreddit = row['subreddit']
                    parent_data = find_parent(parent_id)

                    if score>=2:
                        existing_comment_score = find_existing_comment_score(parent_id) 
                        if existing_comment_score:
                            if score > existing_comment_score:
                                if acceptable_comment(body):
                                    sql_insert_replace_comment(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                        else:
                            if acceptable_comment(body):
                                if parent_data:
                                    sql_insert_has_parent(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                                    paired_rows+=1
                                else:
                                    sql_insert_no_parent(comment_id, parent_id, body, subreddit, created_utc, score)
                except Exception as e:
                            print(str(e)) 

            if row_cnt % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_cnt, paired_rows, str(datetime.now())))

            if row_cnt > start_row:
                if row_cnt % cleanup == 0:
                    print("Cleanin up!")
                    sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                    c.execute(sql)
                    connection.commit()
                    c.execute("VACUUM")
                    connection.commit()


Total Rows Read: 100000, Paired Rows: 4119, Time: 2020-09-06 08:19:10.511261
